Vamos ahora con el feature engineering, en el notebook "split_data" logramos eliminar las columnas que no eran tan escenciales para nuestro proyecto, vamos a continuar explorando que ajustes necesitan nuestros datos para estar en condiciones de de usarse en entrenamiento de modelos.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import os

In [2]:
os.chdir('/content/drive/MyDrive/data_fraud')


In [3]:
train_data = pd.read_csv('train.csv')

In [4]:
# repasemos nuestros datos nuevamente
# Para mostrar todas las columnas
pd.set_option('display.max_columns', None)

train_data.sample(5)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
117113,3159114,0,3715095,57.950,W,4663,490.0,150.0,visa,166.0,debit,272.0,87.0,11.0,NaN,yahoo.com,NaN,114.0,102.0,0.0,0.0,128.0,82.0,0.0,0.0,66.0,0.0,84.0,0.0,399.0,82.0,163.0,163.0,7.0,497.0,7.0,NaN,NaN,NaN,NaN,8.0,130.0,NaN,NaN,NaN,497.0,T,F,F,NaN,NaN,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,45.450001,0.0,0.0,45.450001,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,71.400002,0.0,0.0,71.400002,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22314,3323349,0,8279740,507.950,W,9692,174.0,150.0,visa

Vamos a separar nuestra variable a predecir

In [5]:
# y_train sera nuestra variable a predecir
y_train = train_data['isFraud']
x_train = train_data.drop('isFraud', axis=1)

Usemos nuestra función drop_columns que elaboramos en el notebook drop_columns.

In [6]:
def drop_columns(data):
  """
  Limpia el dataset eliminando columnas predefinidas.

  Args:
    data: DataFrame original a limpiar.

  Returns:
    DataFrame limpio.
  """

  # Columnas con alta cardinalidad
  high_cardinality_cols = ['P_emaildomain', 'R_emaildomain', 'id_30', 'id_31', 'id_33', 'DeviceInfo']

  # Columnas con muchos valores nulos (umbral 50%)
  null_cols = ['dist1', 'dist2', 'D2', 'D5', 'D6', 'D7', 'D8',
               'D9', 'D11', 'D12', 'D13', 'D14', 'M1', 'M2', 'M3', 'M5', 'M7',
               'M8', 'M9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9',
               'V10', 'V11', 'V138', 'V139', 'V140', 'V141', 'V142', 'V143', 'V144',
               'V145', 'V146', 'V147', 'V148', 'V149', 'V150', 'V151', 'V152', 'V153',
               'V154', 'V155', 'V156', 'V157', 'V158', 'V159', 'V160', 'V161', 'V162',
               'V163', 'V164', 'V165', 'V166', 'V167', 'V168', 'V169', 'V170', 'V171',
               'V172', 'V173', 'V174', 'V175', 'V176', 'V177', 'V178', 'V179', 'V180',
               'V181', 'V182', 'V183', 'V184', 'V185', 'V186', 'V187', 'V188', 'V189',
               'V190', 'V191', 'V192', 'V193', 'V194', 'V195', 'V196', 'V197', 'V198',
               'V199', 'V200', 'V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207',
               'V208', 'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216',
               'V217', 'V218', 'V219', 'V220', 'V221', 'V222', 'V223', 'V224', 'V225',
               'V226', 'V227', 'V228', 'V229', 'V230', 'V231', 'V232', 'V233', 'V234',
               'V235', 'V236', 'V237', 'V238', 'V239', 'V240', 'V241', 'V242', 'V243',
               'V244', 'V245', 'V246', 'V247', 'V248', 'V249', 'V250', 'V251', 'V252',
               'V253', 'V254', 'V255', 'V256', 'V257', 'V258', 'V259', 'V260', 'V261',
               'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V269', 'V270',
               'V271', 'V272', 'V273', 'V274', 'V275', 'V276', 'V277', 'V278', 'V322',
               'V323', 'V324', 'V325', 'V326', 'V327', 'V328', 'V329', 'V330', 'V331',
               'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338', 'V339', 'id_01',
               'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09',
               'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17',
               'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25',
               'id_26', 'id_27', 'id_28', 'id_29', 'id_32',
               'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType']
  # Columnas constantes
  constant_cols = ['V305']

  # Columnas altamente correlacionadas
  corr_cols = {'V62', 'V84', 'C2', 'C11', 'V302', 'C14', 'V45', 'V301', 'V65', 'V280',
            'V279', 'V299', 'V308', 'V76', 'C12', 'V309', 'V42', 'C6', 'C9', 'V64',
            'V73', 'V33', 'V134', 'V137', 'V93', 'V94', 'V28', 'V90', 'V38', 'V125',
            'V136', 'V119', 'V307', 'V30', 'V79', 'V60', 'V24', 'V113', 'V128', 'V48',
            'V34', 'V22', 'V70', 'V91', 'V133', 'V291', 'V40', 'V97', 'V285', 'V292',
            'C4', 'V36', 'V49', 'V105', 'V87', 'V294', 'V32', 'V284', 'V109', 'V306',
            'V289', 'V297', 'V106', 'V124', 'V88', 'V102', 'V89', 'V31', 'V316', 'V16',
            'V63', 'V13', 'V295', 'V101', 'V83', 'V311', 'C10', 'V131', 'V72', 'V43',
            'V96', 'C13', 'V20', 'V69', 'V112', 'V21', 'V100', 'V318', 'V54', 'V111',
            'V123', 'V317', 'V303', 'V57', 'V298', 'V18', 'V287', 'V315', 'C7', 'V59',
            'V15', 'V52', 'V58', 'V118', 'V320', 'V110', 'V78', 'V85', 'V321', 'V312',
            'V92', 'V50', 'TransactionDT', 'V127', 'V122', 'V304', 'V81', 'V71', 'V80',
            'V74', 'V26', 'V116', 'V293', 'V103', 'V310', 'C8', 'V132', 'V51', 'V114',
            'V296', 'V68', 'V126'}

  # Columnas no predictivas
  non_predictive_cols = ['TransactionID']

  # Combinar todas las columnas a eliminar
  all_cols_to_drop = high_cardinality_cols + null_cols + constant_cols + list(corr_cols) + non_predictive_cols

  # Eliminar las columnas
  data = data.drop(columns=all_cols_to_drop)

  return data

In [7]:
train_data_drop = pd.DataFrame(drop_columns(x_train))

In [8]:
train_data_drop.sample(5)

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,C1,C3,C5,D1,D3,D4,D10,D15,M4,M6,V12,V14,V17,V19,V23,V25,V27,V29,V35,V37,V39,V41,V44,V46,V47,V53,V55,V56,V61,V66,V67,V75,V77,V82,V86,V95,V98,V99,V104,V107,V108,V115,V117,V120,V121,V129,V130,V135,V281,V282,V283,V286,V288,V290,V300,V313,V314,V319
14928,209.950,W,7676,512.0,150.0,visa,226.0,debit,299.0,87.0,2.0,0.0,1.0,228.0,17.0,604.0,604.0,322.0,NaN,F,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,205.949997,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19434,64.655,C,15885,545.0,185.0,visa,138.0,debit,NaN,NaN,1.0,0.0,0.0,221.0,221.0,221.0,0.0,221.0,M0,NaN,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
45894,46.500,W,11897,321.0,150.0,visa,226.0,credit,433.0,87.0,2.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,M0,T,0.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,2.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.000000,46.5,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
57042,83.500,W,2772,512.0,150.0,visa,226.0,debit,269.0,87.0,2.0,0.0,0.0,0.0,NaN,201.0,565.0,0.0,M0,F,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.000000,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
60694,107.950,W,16132,111.0,150.0,visa,226.0,debit,299.0,87.0,1.0,0.0,1.0,629.0,29.0,NaN,189.0,651.0,NaN,F,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,107.949997,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [9]:
# Identificamos las columnas numéricas
numeric_features = [col for col in train_data_drop.columns if train_data_drop[col].dtype in ['int64', 'float64']]



In [10]:
#veamos que valores tienen las variables numericas, para ver el rengo de sus datos
train_data_drop[numeric_features].describe()

,TransactionAmt,card1,card2,card3,card5,addr1,addr2,C1,C3,C5,D1,D3,D4,D10,D15,V12,V14,V17,V19,V23,V25,V27,V29,V35,V37,V39,V41,V44,V46,V47,V53,V55,V56,V61,V66,V67,V75,V77,V82,V86,V95,V98,V99,V104,V107,V108,V115,V117,V120,V121,V129,V130,V135,V281,V282,V283,V286,V288,V290,V300,V313,V314,V319
count,132397.000000,132397.000000,130339.000000,132072.000000,131423.000000,115421.000000,115421.000000,132397.000000,132397.000000,132397.000000,132086.000000,72480.000000,94540.000000,114757.000000,111876.000000,114744.000000,114744.000000,114744.000000,114744.000000,114744.000000,114744.000000,114744.000000,114744.000000,94533.000000,94533.000000,94533.000000,94533.000000,94533.000000,94533.000000,94533.000000,114277.000000,114277.000000,114277.000000,114277.000000,114277.000000,114277.000000,111864.000000,111864.000000,111864.000000,111864.000000,132325.000000,132325.000000,132325.000000,132325.000000,132325.000000,132325.000000,132325.000000,132325.000000,132325.000000,132325.000000,132325.000000,132325.000000,132325.000000,132086.000000,132086.000000,132086.000000,132395.000000,132086.000000,132395.000000,132086.000000,132086.000000,132086.000000,132395.000000
mean,136.140832,9902.702418,362.793592,153.791788,198.883027,291.074311,86.770562,15.932559,0.005733,5.345265,90.168890,27.540218,135.524836,119.690041,158.308270,0.550573,0.999486,0.155921,0.821524,1.045554,0.978073,0.000749,0.367740,0.528112,1.142680,0.197338,0.999281,1.134620,1.028403,1.050448,0.569353,1.073514,1.135128,0.836809,0.983584,1.003500,0.534953,1.113415,0.852786,1.097905,1.000348,0.064122,0.875330,0.086922,0.999645,1.007149,1.036070,1.001020,1.001345,1.004746,9.357044,92.263205,16.435004,0.098436,0.848856,1.049226,0.031602,0.188832,1.116183,0.047454,22.759329,45.565199,18.321922
std,232.045614,4892.353200,157.934841,12.039052,41.560747,101.895958,2.908836,147.437441,0.157653,25.310308,154.556373,61.496019,189.224408,180.922217,201.180869,0.513762,0.022670,0.404642,0.425784,0.319389,0.190060,0.027683,0.505022,0.517053,0.898168,0.515363,0.026811,0.954453,0.195492,0.288689,0.515550,0.404798,0.774674,0.440555,0.227475,0.264762,0.516858,0.668964,0.426914,0.605038,20.196394,0.289276,2.650674,0.637387,0.018843,0.101426,0.207016,0.041588,0.045981,0.070679,114.063481,312.855997,169.947662,0.548997,0.958323,1.718524,0.190325,0.437173,0.894206,0.299304,100.913449,178.869620,336.221129
min,0.292000,1001.000000,100.000000,100.000000,100.000000,100.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-83.000000,0.000000,-83.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,42.950000,6019.000000,214.000000,150.000000,166.000000,204.000000,87.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,69.000000,9678.000000,361.000000,150.000000,226.000000,299.000000,87.000000,1.000000,0.000000,0.000000,2.000000,7.000000,21.000000,11.000000,42.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1

In [11]:
# tambien las categoricas
categorical_features = [col for col in train_data_drop.columns if train_data_drop[col].dtype == 'object']

Ahora debemos pensar en como imputar los valores nulos, mientras que para las variables numericas podemos sustituir los valores con la media, la mediana, etc., para las variables categóricas debemos usar valores como la variable más frecuente, o introducir valores como 'missig' para representar esos valores nulos, veamos como se comportan.

In [12]:
train_data_drop[categorical_features].sample(10)

,ProductCD,card4,card6,M4,M6
122166,W,mastercard,credit,NaN,F
104242,W,visa,credit,NaN,T
127266,W,mastercard,debit,M0,T
88487,W,visa,credit,M0,T
39901,H,visa,credit,NaN,NaN
118900,C,visa,debit,M2,NaN
35480,W,mastercard,debit,NaN,F
11507,W,visa,debit,M0,F
66850,W,visa,debit,NaN,T
86615,W,mastercard,debit,NaN,T


No se puede ver un orden de jerarquía inherente en las variables categóricas, por lo que lo mejor es imputarlas con el valor más frecuente, para que mantengan su naturaleza categórica nominal.

Posteriormente podemos utilizar OneHotEncoder para codificar nuestras variables, ya que es ideal en casos con variables de baja cardinalidad, que son las que hemos dejado en nuestros datos; el OneHotEncoder también debe alicar para la variable 'ProductCD', así que debemos tenerlo en cuenta.

Respecto a las variables númericas, usar la media o la mediana, quiza sean las mejores opciones.

In [13]:
# Veamos un ejemplo con de como seria esto con la variable c1 y la variable M4
example = train_data_drop[['D15', 'M4']]
example.head(15)

,D15,M4
0,0.0,M2
1,NaN,NaN
2,52.0,NaN
3,14.0,NaN
4,0.0,M0
5,98.0,NaN
6,108.0,M0
7,509.0,NaN
8,0.0,NaN
9,0.0,M2


In [14]:
#aplicamos un Simple Imputer a nuestra variable C3 usando la moda
imputer = SimpleImputer(strategy='median')
example['D15'] = imputer.fit_transform(example[['D15']])

<ipython-input-14-9898c4f63c81>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  example['D15'] = imputer.fit_transform(example[['D15']])


In [15]:
#ahora un Simple Imputer a la variable M4 y posteriormente un OneHotEncoder
imputer = SimpleImputer(strategy='most_frequent')
example['M4'] = imputer.fit_transform(example[['M4']]).ravel()

<ipython-input-15-76c44b0d3609>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  example['M4'] = imputer.fit_transform(example[['M4']]).ravel()


In [16]:
#ahora veamos como quedo nuestro archivo example
example.isnull().sum()

,0
D15,0
M4,0


Y listo, faltaría aplicar OneHotEncoder a las categóricas, para que ahora sí trabajemos con puros números.

In [17]:
#aplicamos OneHotEncoder a la variable M4
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_M4 = encoder.fit_transform(example[['M4']])

# lo volvemos a unir en formato de dataframe con la variable C3
example = pd.concat([example, pd.DataFrame(encoded_M4.toarray(), columns=encoder.get_feature_names_out(['M4']))], axis=1)

In [18]:
example.head(15)

,D15,M4,M4_M0,M4_M1,M4_M2
0,0.0,M2,0.0,0.0,1.0
1,42.0,M0,1.0,0.0,0.0
2,52.0,M0,1.0,0.0,0.0
3,14.0,M0,1.0,0.0,0.0
4,0.0,M0,1.0,0.0,0.0
5,98.0,M0,1.0,0.0,0.0
6,108.0,M0,1.0,0.0,0.0
7,509.0,M0,1.0,0.0,0.0
8,0.0,M0,1.0,0.0,0.0
9,0.0,M2,0.0,0.0,1.0


Y vemos que funciona, no solo ya no tenemos nulos en la variable númerica ni en la categórica, sino que ahora la categórica se ha codificado conéxito, estaría casi lista para para por un algoritmo de entrenamiento.

Sin embargo, imputar valores con la mediana no simpre es bueno en variables numericas, especialmente cuando hay muchos nulos, vamos a ver si hay variables queramos imputar con media.

In [19]:
# Diccionario para guardar el método de imputación
imputation_method = {}

#importamos skew que sirve para calcular la asimetria de las variables numericas
from scipy.stats import skew

# Decidimos la imputación con media o mediana
for feature in numeric_features:
    # Calculamos skewness para cada variable
    skewness = skew(train_data_drop[feature].dropna())
    if abs(skewness) < 0.5:
        imputation_method[feature] = 'mean'
    else:
        imputation_method[feature] = 'median'

# Mostrar el método de imputación para cada variable
for feature, method in imputation_method.items():
    # hacemos una lista de las varibles quedeben ser imputadas con mean
    imputacion_numerica_media = [feature for feature, method in imputation_method.items() if method == 'mean']
    # ahora para el resto
    imputacion_numerica_mediana = [feature for feature, method in imputation_method.items() if method == 'median']


In [20]:
print('Variables a imputar con media:', imputacion_numerica_media)
print('Variables a imputar con mediana: ',imputacion_numerica_mediana)

Variables a imputar con media: ['card1', 'card2', 'addr1', 'V12', 'V35', 'V53', 'V61', 'V66', 'V75']
Variables a imputar con mediana:  ['TransactionAmt', 'card3', 'card5', 'addr2', 'C1', 'C3', 'C5', 'D1', 'D3', 'D4', 'D10', 'D15', 'V14', 'V17', 'V19', 'V23', 'V25', 'V27', 'V29', 'V37', 'V39', 'V41', 'V44', 'V46', 'V47', 'V55', 'V56', 'V67', 'V77', 'V82', 'V86', 'V95', 'V98', 'V99', 'V104', 'V107', 'V108', 'V115', 'V117', 'V120', 'V121', 'V129', 'V130', 'V135', 'V281', 'V282', 'V283', 'V286', 'V288', 'V290', 'V300', 'V313', 'V314', 'V319']


Por otro lado, debemos pensar en el escalamiento, las varibales categóricas no se deben escalar, porque la codificación con OneHotEncoder las deja ya en unos y ceros, que estan perfectos para los modelos de machine learning, sin embargo, las variables numericas y su escalado dependen de el tipo de modelo que elijamos. Como nuestra intención es realizar diferentes tipos de modelos, debemos hacer un pipeline en donde escalemos todas nuestras variables  númericas que usaremos para modelos como Regresión logistica y otro en donde no se escalen, nos servira para modelos como Random Forest.

Sin embargo en ambos casos es probable que necesitemos otro tipo de transformacion, por ejemplo una transformación logaritmica para variables con alta asimetría positiva y alta curtosis, una transformación de raíz cuadrada para asimetría moderada y una inversa para manejar una alta asimetría negativa.

In [21]:
# Ahora hacemos un algoritmo para determinar que variables numericas deben tranformarse y cuales deben dejarse asi dependiendo de su distribucion

transformacion_logaritmica = []
transformacion_raiz_cuadrada = []
transformacion_inversa = []
transformacion_none = []

from scipy.stats import skew, kurtosis

for feature in numeric_features:
    skewness = skew(train_data_drop[feature].dropna())
    kurt = kurtosis(train_data_drop[feature].dropna())

    print(f"Variable {feature}: Asimetría = {skewness}, Curtosis = {kurt}")

    if skewness < -2:
        transformacion_inversa.append(feature)
    elif kurt > 4 or skewness > 2:
        transformacion_logaritmica.append(feature)
    elif 1 < abs(skewness) < 2:
        transformacion_raiz_cuadrada.append(feature)
    else:
        transformacion_none.append(feature)

Variable TransactionAmt: Asimetría = 6.826723678093948, Curtosis = 70.10936453329202
Variable card1: Asimetría = -0.04119673655136268, Curtosis = -1.1324773773858712
Variable card2: Asimetría = -0.2021496202073679, Curtosis = -1.335201905716821
Variable card3: Asimetría = 1.8522826047672678, Curtosis = 4.7395347809501605
Variable card5: Asimetría = -1.2026302552223977, Curtosis = -0.11375419711677681
Variable addr1: Asimetría = 0.36838932084615994, Curtosis = -0.5126472888677407
Variable addr2: Asimetría = -13.519698576575275, Curtosis = 224.85706556874314
Variable C1: Asimetría = 21.855330170809708, Curtosis = 553.7803220936153
Variable C3: Asimetría = 95.17402426986888, Curtosis = 12231.323753024011
Variable C5: Asimetría = 5.924194732608398, Curtosis = 37.69814541887424
Variable D1: Asimetría = 1.8795273863432087, Curtosis = 2.509716383021436
Variable D3: Asimetría = 4.562130391805705, Curtosis = 25.158017502195147
Variable D4: Asimetría = 1.2180064851524102, Curtosis = 0.0818979215

In [22]:
print("Transformación Logaritmica:", transformacion_logaritmica)
print("Transformación Raíz Cuadrada:", transformacion_raiz_cuadrada)
print("Transformación Inversa:", transformacion_inversa)
print("Sin Transformación:", transformacion_none)

Transformación Logaritmica: ['TransactionAmt', 'card3', 'C1', 'C3', 'C5', 'D3', 'V17', 'V23', 'V27', 'V37', 'V39', 'V44', 'V46', 'V47', 'V55', 'V56', 'V66', 'V67', 'V77', 'V86', 'V95', 'V98', 'V99', 'V104', 'V108', 'V115', 'V117', 'V120', 'V121', 'V129', 'V130', 'V135', 'V281', 'V282', 'V283', 'V286', 'V288', 'V290', 'V300', 'V313', 'V314', 'V319']
Transformación Raíz Cuadrada: ['card5', 'D1', 'D4', 'D10', 'D15']
Transformación Inversa: ['addr2', 'V14', 'V25', 'V41', 'V107']
Sin Transformación: ['card1', 'card2', 'addr1', 'V12', 'V19', 'V29', 'V35', 'V53', 'V61', 'V75', 'V82']


Una vez que hemos determinado todos los ajustes que le haremos a las diferentes variables, debemos agruparlas columnas a las que le haremos exactamente los mismos procediemientos. Veamos cuantas listas de datos hemos creado.

In [23]:
print('variables_categoricas:', categorical_features)
print('variables_numericas: ', numeric_features)
print('imputacion_numerica_media:', imputacion_numerica_media)
print('imputacion_numerica_mediana:', imputacion_numerica_mediana)
print('transformacion_logistica:', transformacion_logaritmica)
print('transformacion_raiz_cuadrada:', transformacion_raiz_cuadrada)
print('transformacion_inversa:', transformacion_inversa)
print('transformacion_none:', transformacion_none)

variables_categoricas: ['ProductCD', 'card4', 'card6', 'M4', 'M6']
variables_numericas:  ['TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'C1', 'C3', 'C5', 'D1', 'D3', 'D4', 'D10', 'D15', 'V12', 'V14', 'V17', 'V19', 'V23', 'V25', 'V27', 'V29', 'V35', 'V37', 'V39', 'V41', 'V44', 'V46', 'V47', 'V53', 'V55', 'V56', 'V61', 'V66', 'V67', 'V75', 'V77', 'V82', 'V86', 'V95', 'V98', 'V99', 'V104', 'V107', 'V108', 'V115', 'V117', 'V120', 'V121', 'V129', 'V130', 'V135', 'V281', 'V282', 'V283', 'V286', 'V288', 'V290', 'V300', 'V313', 'V314', 'V319']
imputacion_numerica_media: ['card1', 'card2', 'addr1', 'V12', 'V35', 'V53', 'V61', 'V66', 'V75']
imputacion_numerica_mediana: ['TransactionAmt', 'card3', 'card5', 'addr2', 'C1', 'C3', 'C5', 'D1', 'D3', 'D4', 'D10', 'D15', 'V14', 'V17', 'V19', 'V23', 'V25', 'V27', 'V29', 'V37', 'V39', 'V41', 'V44', 'V46', 'V47', 'V55', 'V56', 'V67', 'V77', 'V82', 'V86', 'V95', 'V98', 'V99', 'V104', 'V107', 'V108', 'V115', 'V117', 'V120', 'V121', 

In [24]:
# Vamos a establecer bien nuestro plan, la variables categoricas se vana imputar con el valor más fecuente despues solo necesitan OneHotEncoder
# Por otro lado vamos con las variables numericas, seleccionemos las que se imputaran con la media, mediana, posteriormente con transformacion similar,
# para que al final todas puedan escalarse, o no dependiendo el caso
media_logaritmica = []
mediana_logaritmica = []
media_raiz_cuadrada = []
mediana_raiz_cuadrada = []
media_inversa = []
mediana_inversa = []
media_none = []
mediana_none = []

for feature in numeric_features:
    if feature in imputacion_numerica_media:
        if feature in transformacion_logaritmica:
            media_logaritmica.append(feature)
        if feature in transformacion_raiz_cuadrada:
            media_raiz_cuadrada.append(feature)
        if feature in transformacion_inversa:
            media_inversa.append(feature)
        if feature in transformacion_none:
            media_none.append(feature)
    if feature in imputacion_numerica_mediana:
        if feature in transformacion_logaritmica:
            mediana_logaritmica.append(feature)
        if feature in transformacion_raiz_cuadrada:
            mediana_raiz_cuadrada.append(feature)
        if feature in transformacion_inversa:
            mediana_inversa.append(feature)
        if feature in transformacion_none:
            mediana_none.append(feature)



Ahora si, sabemos que necesitamos transformar 6 conjuntos de datos de variables numericas y un conjunto de datos de variables categóricas.

1.- Variables numericas, con imputación por la media, tranformación logaritmica y escalado.

2.- Variables numéricas con imputación por la mediana, tranformación logaritmica y escalado.

3.- Variables númericas con imputación con mediana, transformación por raiz cuadrada y escalado.

4.-Variables númericas con imputación con mediana, transformación inversa y escalado.

5.- Variables númericas con imputación con la media y escalado

6.- Una variable númerica con imputación con la mediana y escalado.

7.- Variables categóricas con imputación por el valor más frecuente y codificación con OneHotEncoder.

In [25]:
print('media_logaritmica:', media_logaritmica, len(media_logaritmica))
print('mediana_logaritmica:', mediana_logaritmica, len(mediana_logaritmica))
print('media_raiz_cuadrada:', media_raiz_cuadrada, len(media_raiz_cuadrada))
print('mediana_raiz_cuadrada:', mediana_raiz_cuadrada, len(mediana_raiz_cuadrada))
print('media_inversa:', media_inversa, len(media_inversa))
print('mediana_inversa:', mediana_inversa, len(mediana_inversa))
print('media_none:', media_none, len(media_none))
print('mediana_none:', mediana_none, len(mediana_none))
print(len(media_inversa)+len(mediana_inversa)+len(media_logaritmica)+len(mediana_logaritmica)+len(media_raiz_cuadrada)+len(mediana_raiz_cuadrada)+len(media_none)+len(mediana_none))

media_logaritmica: ['V66'] 1
mediana_logaritmica: ['TransactionAmt', 'card3', 'C1', 'C3', 'C5', 'D3', 'V17', 'V23', 'V27', 'V37', 'V39', 'V44', 'V46', 'V47', 'V55', 'V56', 'V67', 'V77', 'V86', 'V95', 'V98', 'V99', 'V104', 'V108', 'V115', 'V117', 'V120', 'V121', 'V129', 'V130', 'V135', 'V281', 'V282', 'V283', 'V286', 'V288', 'V290', 'V300', 'V313', 'V314', 'V319'] 41
media_raiz_cuadrada: [] 0
mediana_raiz_cuadrada: ['card5', 'D1', 'D4', 'D10', 'D15'] 5
media_inversa: [] 0
mediana_inversa: ['addr2', 'V14', 'V25', 'V41', 'V107'] 5
media_none: ['card1', 'card2', 'addr1', 'V12', 'V35', 'V53', 'V61', 'V75'] 8
mediana_none: ['V19', 'V29', 'V82'] 3
63


Además debemos agregarle la dunción que creamos en el notebook 'split_data', para que todo quede en un solo pipeline.

In [26]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

In [27]:
#supongamos que aquí llamamos a nuestros datos de entrenamiento
train_data = pd.read_csv('train.csv')

# Definimos la función para eliminar columnas
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Columnas con alta cardinalidad
        high_cardinality_cols = ['P_emaildomain', 'R_emaildomain', 'id_30', 'id_31', 'id_33', 'DeviceInfo']

        # Columnas con muchos valores nulos (umbral 50%)
        null_cols = ['dist1', 'dist2', 'D2', 'D5', 'D6', 'D7', 'D8', 'D9', 'D11', 'D12', 'D13', 'D14', 'M1', 'M2', 'M3', 'M5', 'M7',
                    'M8', 'M9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V138', 'V139', 'V140', 'V141',
                    'V142', 'V143', 'V144', 'V145', 'V146', 'V147', 'V148', 'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155',
                    'V156', 'V157', 'V158', 'V159', 'V160', 'V161', 'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V168', 'V169',
                    'V170', 'V171', 'V172', 'V173', 'V174', 'V175', 'V176', 'V177', 'V178', 'V179', 'V180', 'V181', 'V182', 'V183',
                    'V184', 'V185', 'V186', 'V187', 'V188', 'V189', 'V190', 'V191', 'V192', 'V193', 'V194', 'V195', 'V196', 'V197',
                    'V198', 'V199', 'V200', 'V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V211',
                    'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V218', 'V219', 'V220', 'V221', 'V222', 'V223', 'V224', 'V225',
                    'V226', 'V227', 'V228', 'V229', 'V230', 'V231', 'V232', 'V233', 'V234', 'V235', 'V236', 'V237', 'V238', 'V239',
                    'V240', 'V241', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V248', 'V249', 'V250', 'V251', 'V252', 'V253',
                    'V254', 'V255', 'V256', 'V257', 'V258', 'V259', 'V260', 'V261', 'V262', 'V263', 'V264', 'V265', 'V266', 'V267',
                    'V268', 'V269', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276', 'V277', 'V278', 'V322', 'V323', 'V324',
                    'V325', 'V326', 'V327', 'V328', 'V329', 'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
                    'V339', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12',
                    'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25',
                    'id_26', 'id_27', 'id_28', 'id_29', 'id_32', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType']

        # Columnas constantes
        constant_cols = ['V305']

        # Columnas altamente correlacionadas
        corr_cols = {'V62', 'V84', 'C2', 'C11', 'V302', 'C14', 'V45', 'V301', 'V65', 'V280', 'V279', 'V299', 'V308', 'V76', 'C12',
                    'V309', 'V42', 'C6', 'C9', 'V64', 'V73', 'V33', 'V134', 'V137', 'V93', 'V94', 'V28', 'V90', 'V38', 'V125', 'V136',
                    'V119', 'V307', 'V30', 'V79', 'V60', 'V24', 'V113', 'V128', 'V48', 'V34', 'V22', 'V70', 'V91', 'V133', 'V291',
                    'V40', 'V97', 'V285', 'V292', 'C4', 'V36', 'V49', 'V105', 'V87', 'V294', 'V32', 'V284', 'V109', 'V306', 'V289',
                    'V297', 'V106', 'V124', 'V88', 'V102', 'V89', 'V31', 'V316', 'V16', 'V63', 'V13', 'V295', 'V101', 'V83', 'V311',
                    'C10', 'V131', 'V72', 'V43', 'V96', 'C13', 'V20', 'V69', 'V112', 'V21', 'V100', 'V318', 'V54', 'V111', 'V123',
                    'V317', 'V303', 'V57', 'V298', 'V18', 'V287', 'V315', 'C7', 'V59', 'V15', 'V52', 'V58', 'V118', 'V320', 'V110',
                    'V78', 'V85', 'V321', 'V312', 'V92', 'V50', 'TransactionDT', 'V127', 'V122', 'V304', 'V81', 'V71', 'V80', 'V74',
                    'V26', 'V116', 'V293', 'V103', 'V310', 'C8', 'V132', 'V51', 'V114', 'V296', 'V68', 'V126'}

        # Columnas no predictivas
        non_predictive_cols = ['TransactionID']

        # Combinamos todas las columnas a eliminar
        all_cols_to_drop = high_cardinality_cols + null_cols + constant_cols + list(corr_cols) + non_predictive_cols

        # Eliminamos las columnas
        X = X.drop(columns=all_cols_to_drop)

        return X

# Funciones de transformación
def log_transform(x):
    return np.log1p(x)

def inverse_log_transform(x):
    x_reflected = x.max() - x  # Reflejamos valores
    return np.log1p(x_reflected)

def sqrt_transform(x):
    return np.sqrt(x)

# Columnas categóricas y numéricas
categorical_features = ['ProductCD', 'card4', 'card6', 'M4', 'M6']
numeric_mean_log_cols = ['V66']
numeric_median_log_cols = ['TransactionAmt', 'card3', 'C1', 'C3', 'C5', 'D3', 'V17',
                           'V23', 'V27', 'V37', 'V39', 'V44', 'V46', 'V47', 'V55', 'V56',
                           'V67', 'V77', 'V86', 'V95', 'V98', 'V99', 'V104', 'V108', 'V115',
                           'V117', 'V120', 'V121', 'V129', 'V130', 'V135', 'V281', 'V282', 'V283',
                           'V286', 'V288', 'V290', 'V300', 'V313', 'V314', 'V319']
numeric_median_sqrt_cols = ['card5', 'D1', 'D4', 'D10', 'D15']
numeric_median_inv_cols = ['addr2', 'V14', 'V25', 'V41', 'V107']
numeric_mean_cols = ['card1', 'card2', 'addr1', 'V12', 'V35', 'V53', 'V61', 'V75']
numeric_median_cols = ['V19', 'V29', 'V82']

# Creamos los transformadores
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numeric_mean_log_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('log', FunctionTransformer(log_transform)),
    ('scaler', StandardScaler())
])

numeric_median_log_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('log', FunctionTransformer(log_transform)),
    ('scaler', StandardScaler())
])

numeric_median_sqrt_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('sqrt', FunctionTransformer(sqrt_transform)),
    ('scaler', StandardScaler())
])

numeric_median_inv_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('inv', FunctionTransformer(inverse_log_transform)),
    ('scaler', StandardScaler())
])

numeric_mean_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

numeric_median_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Unir los transformadores en un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num_mean_log', numeric_mean_log_transformer, numeric_mean_log_cols),
        ('num_median_log', numeric_median_log_transformer, numeric_median_log_cols),
        ('num_median_sqrt', numeric_median_sqrt_transformer, numeric_median_sqrt_cols),
        ('num_median_inv', numeric_median_inv_transformer, numeric_median_inv_cols),
        ('num_mean', numeric_mean_transformer, numeric_mean_cols),
        ('num_median', numeric_median_transformer, numeric_median_cols),
        ('cat', categorical_transformer, categorical_features)
    ])

# Crear el pipeline completo
pipeline = Pipeline(steps=[('drop_columns', DropColumns()),
                           ('preprocessor', preprocessor)])

# Separar el target y las características
X_train = train_data.drop(columns=['isFraud'])
y_train = train_data['isFraud']

# Ajustar el pipeline y transformar los datos
X_train_prepared = pipeline.fit_transform(X_train)

# Ahora X_train_prepared esta listo para ser usado en los algoritmos de entrenamiento

<ipython-input-27-aa3dee90f4f6>:68: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(x)


Nuestros datos han dejado de ser un DataFrame, pero podemos regresrlo a uno para ver como quedo nuestro x_train_prepared.

In [28]:
# convertimos X_train_prepared a un dataframe
x_train_example = pd.DataFrame(X_train_prepared)
x_train_example.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80
0,0.120344,-2.648679,2.539107,-0.549829,-0.060386,-0.487206,-0.000975,2.499934,-0.145638,-0.025296,-0.195707,2.399497,-0.184260,-0.120915,-0.158337,-0.193257,-0.246742,0.048742,-0.196051,-0.188060,-0.333779,-0.235596,-0.576977,-0.198775,-0.067872,-0.178903,-0.015635,-0.020616,-0.065214,-0.236716,-0.648965,-0.215496,-0.216888,0.410963,0.251651,-0.171908,-0.447545,-0.238052,-0.180084,-0.444124,-0.485073,-0.218805,0.593584,-0.773867,-0.932713,-0.838231,-1.036292,-0.063369,-0.021115,-0.106677,-0.022669,-0.018845,-1.379444,0.288487,0.000000,-1.151144,-1.208761,-1.188702,0.398711,-1.126005e+00,0.385735,-0.655127,0.314087,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.120344,-0.518860,-0.298775,3.378762,-0.060386,3.998267,-0.113010,-0.379096,-0.145638,-0.025296,-0.195707,-0.363961,-0.184260,-0.120915,-0.158337,-0.193257,-0.246742,0.048742,-0.196051,-0.188060,-0.333779,-0.235596,-0.576977,-0.198775,-0.067872,-0.178903,-0.015635,-0.020616,-0.065214,-0.236716,-0.648965,-0.215496,-0.216888,-1.208152,-1.131244,-0.171908,-0.447545,-0.238052,-0.180084,-0.444124,-0.485073,-0.218805,0.593584,1.346631,-0.314602,-0.054032,-0.248322,-0.063369,-0.021115,-0.106677,-0.022669,-0.018845,0.731409,0.524604,-1.766619,0.939665,0.000000,0.899108,2.841927,-2.336871e-16,0.385735,1.400444,0.314087,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.120344,-0.323185,-0.298775,3.320400,-0.060386,3.838128,-0.113010,-0.379096,4.138419,-0.025296,-0.195707,-0.363961,-0.184260,-0.120915,-0.158337,-0.193257,-0.246742,0.048742,-0.196051,2.705877,-0.333779,-0.235596,2.077018,-0.198775,-0.067872,-0.178903,-0.015635,-0.020616,-0.065214,-0.236716,1.666531,-0.215496,-0.216888,-1.208152,-1.131244,-0.171908,-0.447545,-0.238052,-0.180084,-0.444124,2.062662,-0.218805,0.593584,0.178189,-0.314602,0.079122,-0.159520,-0.063369,-0.021115,-0.106677,-0.022669,-0.018845,0.031539,-0.879336,0.251481,0.939665,0.000000,-1.188702,-2.044504,9.788615e-01,-2.108045,1.400444,-2.211063,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.120344,0.332963,-0.298775,0.692743,-0.060386,1.307809,0.526436,-0.379096,-0.145638,-0.025296,-0.195707,-0.363961,-0.184260,-0.120915,-0.158337,-0.193257,-0.246742,0.048742,-0.196051,-0.188060,-0.333779,-0.235596,-0.576977,-0.198775,-0.067872,-0.178903,-0.015635,-0.020616,-0.065214,-0.236716,-0.648965,-0.215496,-0.216888,-1.208152,-1.131244,-0.171908,-0.447545,-0.238052,-0.180084,-0.444124,-0.485073,-0.218805,0.635346,0.168809,-0.314602,0.052267,-0.581357,-0.063369,-0.021115,-0.106677,-0.022669,-0.018845,-1.391708,-0.553877,-1.051875,0.939665,0.000000,0.899108,-2.044504,9.788615e-01,-2.108045,1.400444,0.314087,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.120344,0.602229,-0.298775,-0.549829,-0.060386,-0.487206,-0.000975,-0.379096,-0.145638,-0.025296,-0.195707,-0.363961,-0.184260,-0.120915,-0.158337,-0.193257,-0.246742,0.048742,-0.196051,-0.188060,-0.333779,-0.235596,-0.576977,-0.198775,-0.067872,-0.178903,-0.015635,-0.020616,-0.065214,-0.236716,-0.648965,-0.215496,-0.216888,0.410963,0.251651,-0.171908,-0.447545,-0.238052,-0.180084,-0.444124,-0.485073,-0.218805,0.121280,-0.773867,-0.932713,-0.838231,-1.036292,-0.063369,-0.021115,-0.106677,-0.022669,-0.018845,-0.405472,-1.077163,0.356591,0.939665,1.080074,0.899108,0.398711,9.788615e-01,0.385735,-0.655127,0.314087,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
5,0.120344,0.516953,-0.298775,0.137680,-0.060386,0.207201,1.661112,-0.379096,-0.145638,-0.025296,-0.195707,-0.363961,-0.184260,-0.120915,-0.158337,-0.193257,-0.246742,0.048742,-0.196051,-0.188060,-0.333779,-0.235596,-0.576977,-0.198775,-0.067872,-0.178903,-0.015635,-0.020616,-0.065214,-0.236716,-0.64

Y listo, nuestro pipeline se encuentra preparado para que lo unamos a un modelo de machine learning y podamos centraranos en las diferentes técnicas de modelado en busca de nuestras mejores predicciones, solo tengamos en cuenta que para modelos como Random Forest debemos quitar el escalado de las variables númericas